#  Data collection & Munging

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd

In [64]:
tree = ET.parse('datasets/new-xml/20160410 FC Utrecht - NEC-new.xml')

root = tree.getroot()

def xmlToDataframe(root):
    all_instances = root[1]
    all_records = []
    headers = []
    for i, child in enumerate(all_instances):
        current_label = 0
        record = []
        for subchild in child:
            label_name = ''
            if subchild.tag == 'label':
                subrecord = []
                label_name = subchild.tag+'.'+str(current_label)
                if label_name not in headers:
                        headers.append(label_name)
                for subsubchild in subchild:
                    subrecord.append(subsubchild.text)
                current_label += 1
                record.append(subrecord)
            else:
                record.append(subchild.text)
            if subchild.tag not in headers and subchild.tag not in 'label':
                headers.append(subchild.tag)
        print(record)
        all_records.append(record)
    #print(headers)
    return pd.DataFrame(all_records, columns=headers)

xmlToDataframe(root)
# print(df.head())
# print(df.tail())

['1', '4.3110428970', '40.9711876731', 'TEG', ['Naar zone', 'M2 TEG'], ['Naar zone', 'L2 TEG'], ['Naar zone', 'M1 TEG'], ['Storen NEC', 'Compact'], ['Storen NEC', 'Druk'], ['Naar zone', 'R1 TEG'], ['Naar zone', 'R2 TEG'], ['Naar zone', 'M1 TEG'], ['Naar zone', 'R1 TEG'], ['Aanvallen TEG', 'Lange pass'], ['Rendement', 'Balwinst']]
['2', '6.2150178417', '15.1347948447', 'M2 TEG', ['Naar zone', 'L2 TEG']]
['3', '6.3110428970', '40.9711876731', 'Start Jaimy', ['Naar zone', 'M2 TEG'], ['Naar zone', 'L2 TEG'], ['Naar zone', 'M1 TEG'], ['Storen NEC', 'Compact'], ['Storen NEC', 'Druk'], ['Naar zone', 'R1 TEG'], ['Naar zone', 'R2 TEG'], ['Naar zone', 'M1 TEG'], ['Naar zone', 'R1 TEG'], ['Aanvallen TEG', 'Lange pass'], ['Rendement', 'Balwinst']]
['4', '6.3110428970', '40.9711876731', 'Verdedigen', ['Naar zone', 'M2 TEG'], ['Naar zone', 'L2 TEG'], ['Naar zone', 'M1 TEG'], ['Storen NEC', 'Compact'], ['Storen NEC', 'Druk'], ['Naar zone', 'R1 TEG'], ['Naar zone', 'R2 TEG'], ['Naar zone', 'M1 TEG'], 

,ID,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,...,label.62,label.63,label.64,label.65,label.66,label.67,label.68,label.69,label.70,label.71
0,1,4.3110428970,40.9711876731,TEG,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
1,2,6.2150178417,15.1347948447,M2 TEG,"[Naar zone, L2 TEG]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,3,6.3110428970,40.9711876731,Start Jaimy,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
3,4,6.3110428970,40.9711876731,Verdedigen,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
4,5,8.2200000000,14.0400000000,AFTRAP WB,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,6,12.1347948447,21.9750148509,L2 TEG,"[Naar zone, M1 TEG]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,7,18.9750148509,26.2629141727,M1 TEG,"[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,8,20.4471341203,40.9711876731,VERD. Compact NEC,"[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]","[Naar zone, M1 TEG]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
8,9,20.4471341203,40.9711876731,VERD. Druk NEC,"[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]","[Naar zone, M1 TEG]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
9,10,23.2629141727,30.9632140377,R1 TEG,"[Naar zone, R2 TEG]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
